In [ ]:
import h5py
import torch
import numpy as np
from matplotlib import pyplot as plt
from calibrate.net.unet import UNet
from calibrate.evaluation.reliability_diagram import ReliabilityDiagram
import torch.nn.functional as F
from sklearn.calibration import calibration_curve
import glob
from tqdm import tqdm

In [ ]:
model = UNet(input_channels=1, num_classes=2)
model_path = '/home/ar88770/MarginLoss/outputs/prostate/unet-ce-adam/20220323-17:16:52-337408/best.pth'
checkpoint = torch.load(model_path)["state_dict"]
checkpoint = dict((key[7:] if "module" in key else key, value)for (key, value) in checkpoint.items())
model.load_state_dict(checkpoint)
model = model.to('cuda:3')

In [ ]:
data_root = '/home/ar88770/MarginLoss/promise/valid'
files = glob.glob('{}/*.h5'.format(data_root))

mask_stack = np.empty([])
pred_stack = np.empty([])

for fpath in tqdm(files):
    
    with h5py.File(fpath, 'r') as hf:
        img = hf['img'][:]
        mask = hf['mask'][:]
        
    imgT = torch.from_numpy(img)
    imgT = imgT.unsqueeze(0).unsqueeze(0)
    imgT = imgT.cuda('cuda:3')
    
    predT = F.softmax(model(imgT.float()),dim=1)[0]
    
    ind = mask == 1
    prednp = predT.detach().cpu().numpy()
    pred_fg = prednp[:,ind]
    pred_fg = pred_fg.T[:,1]
    mask_fg = mask[ind]
    
    pred_stack = np.hstack([pred_stack, pred_fg])
    mask_stack = np.hstack([mask_stack, mask_fg])
    
#     break

In [ ]:
print (pred_stack.shape, mask_stack.shape)

In [ ]:
diagram = ReliabilityDiagram(bins=25, style='curve')

In [ ]:
diagram.plot(pred_fg,mask_fg)